## Problem 3: How many people live under the dominance area of each shopping center? (additional 5 points available!)

Find out how many people live under the dominance area of each shopping center. 

- Check out week 3 materials for reading in population grid from [HSY wfs](https://www.hsy.fi/fi/asiantuntijalle/avoindata/karttapalvelu/Sivut/Avoimet-rajapinnat.aspx).
- Aggregate your dominance areas into a unified geometries using [`dissolve()`](http://geopandas.org/aggregation_with_dissolve.html#dissolve-example) -function in Geopandas before joining with the population data
- Make the spatial join using `intersect`as the condition

You can freely organize your code into the code cells below.



In [1]:
# import libraries
#import glob
import geopandas as gpd
import pandas as pd
import re
import numpy as np
import matplotlib.pyplot as plt
from pyproj import CRS
import requests
import geojson

#### Read and import shopping centers data and population data files

In [128]:
#1 Read shopping center data file (file made in problem 2)
shopping_center_data = gpd.read_file("shopping_center_data.shp")

# Drop all the columns except dominant_service and geometry
shopping_center_data = shopping_center_data[["dominant_s", "geometry"]]

# Rename column dominant_s back to dominant_service
shopping_center_data = shopping_center_data.rename(columns={'dominant_s': 'dominant_service'})

#2 Read population grid data

# Specify the url for web feature service
url = 'https://kartta.hsy.fi/geoserver/wfs'

# Specify parameters (read data in json format).
# Available feature types in this particular data source: http://geo.stat.fi/geoserver/vaestoruutu/wfs?service=wfs&version=2.0.0&request=describeFeatureType
params = dict(service='WFS',
              version='2.0.0',
              request='GetFeature',
              typeName='asuminen_ja_maankaytto:Vaestotietoruudukko_2018',
              outputFormat='json')

# Fetch data from WFS using requests
r = requests.get(url, params=params)

# Create GeoDataFrame from geojson
pop = gpd.GeoDataFrame.from_features(geojson.loads(r.content))

# Drop all the columns exceps asukkaita and geometry
pop = pop[["asukkaita", "geometry"]]

# Rename column asukkaita to pop18
pop = pop.rename(columns={'asukkaita': 'pop18'})

In [4]:

# Create function to get population data
def get_pop_data(year):
    
    # Specify the url for web feature service
    url = 'https://kartta.hsy.fi/geoserver/wfs'
    
    # Specify parameters (read data in json format).
    # Available feature types in this particular data source: http://geo.stat.fi/geoserver/vaestoruutu/wfs?service=wfs&version=2.0.0&request=describeFeatureType
    params = dict(service='WFS', version='2.0.0', request='GetFeature', typeName='asuminen_ja_maankaytto:Vaestoruudukko_'+str(year), outputFormat='json')

    # Fetch data from WFS using requests
    r = requests.get(url, params=params)

    # Create GeoDataFrame from geojson
    pop = gpd.GeoDataFrame.from_features(geojson.loads(r.content))
    
    # Define coordinate reference system
    pop.crs = CRS.from_epsg(3879).to_wkt()
    
    # Drop all the columns exceps asukkaita and geometry
    pop = pop[["index", "asukkaita", "geometry"]]

    # Rename column asukkaita to pop18
    pop = pop.rename(columns={'asukkaita': 'pop'+str(year)})
    
    # Return result geodataframe
    return pop


pop_2010 = get_pop_data(1997)
#pop_2011 = get_pop_data(2011)
#pop_2012 = get_pop_data(2012)
#pop_2013 = get_pop_data(2013)
#pop_2014 = get_pop_data(2014)
pop_2015 = get_pop_data(2015)
pop_2016 = get_pop_data(2016)
pop_2017 = get_pop_data(2017)
pop_2018 = get_pop_data(2018)
pop_2019 = get_pop_data(2019)

pop_2019

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

#### Set coordinate reference system to EPSG 3879 to both geodataframe

In [129]:
# Define crs
pop.crs = CRS.from_epsg(3879).to_wkt()

# Are the layers in the same projection?
print("Is the crs same on boths dataframes? Result:", pop.crs == shopping_center_data.crs)

# Reproject geodata to coordinates to population crs
shopping_center_data = shopping_center_data.to_crs(pop.crs)

# Are the layers now in the same projection?
print("Is the crs same on boths dataframes? Result:", pop.crs == shopping_center_data.crs)

Is the crs same on boths dataframes? Result: False
Is the crs same on boths dataframes? Result: True


#### Aggregate shopping centers dominance areas into a unified geometries

In [130]:
# Aggregate your dominance areas into a unified geometries
shopping_centers_areas = shopping_center_data.dissolve(by='dominant_service')

# Show geodataframe
shopping_centers_areas

,geometry
dominant_service,
pt_r_t_Dixi,"MULTIPOLYGON (((25508886.745 6684266.016, 2550..."
pt_r_t_Forum,"MULTIPOLYGON (((25495548.583 6670352.148, 2549..."
pt_r_t_IsoOmena,"MULTIPOLYGON (((25477492.450 6671806.572, 2547..."
pt_r_t_Itis,"MULTIPOLYGON (((25500759.604 6671760.862, 2550..."
pt_r_t_Jumbo,"MULTIPOLYGON (((25506167.581 6683182.834, 2550..."
pt_r_t_Myyrmanni,"MULTIPOLYGON (((25495351.703 6676850.706, 2549..."
pt_r_t_Ruoholahti,"MULTIPOLYGON (((25487452.020 6673359.116, 2548..."


#### Make the spatial join using intersectas the condition

In [131]:
# Create a spatial join between shopping_center_areas and pop dataframes 
shopping_center_population =  gpd.sjoin(pop, shopping_centers_areas, how="inner", op="intersects")

# Group data by shopping center name and calculate population sum
shopping_center_population = shopping_center_population.groupby("index_right").sum()

# Show dataframe
shopping_center_population

,pop18
index_right,
pt_r_t_Dixi,201962
pt_r_t_Forum,252721
pt_r_t_IsoOmena,174969
pt_r_t_Itis,201066
pt_r_t_Jumbo,69613
pt_r_t_Myyrmanni,203275
pt_r_t_Ruoholahti,83754


#### Count and print number of people in each shopping centers dominant areas

In [132]:
for index, row in shopping_center_population.iterrows():
    name = re.search('pt_r_t_(\D*)', index).group(1)
    print(row["pop18"], "people live in the dominance area of shopping center", name + ".\n")

201962 people live in the dominance area of shopping center Dixi.

252721 people live in the dominance area of shopping center Forum.

174969 people live in the dominance area of shopping center IsoOmena.

201066 people live in the dominance area of shopping center Itis.

69613 people live in the dominance area of shopping center Jumbo.

203275 people live in the dominance area of shopping center Myyrmanni.

83754 people live in the dominance area of shopping center Ruoholahti.



Remember to print the answers in this notebook :)

### Extra bonus task

Repeat problem 2 and 3 for car accessibility! No extra points available for this, but you can for example start thinking if you want to make the final assignment related to this topic :)